# PREPARE
You don't need mongodb to run this notebook. You can just use ConversaionBufferMemmory instead. This is for testing purpose only.

To be able to run the test in this notebook, you need some file in `CompassUTD\_secret\` folder. The folder should be in the same directory as this notebook. The files are:

`google_key.json`: - Google API key for Google Cloud Platform. You can get one by making a service account
`search_secrets.json`: - Google Custom Search API key and search engine ID. You can get one by making a custom search engine with these settings:
```
{
    "GOOGLE_SEARCH_API": "REPlACE_WITH_YOUR_KEY",
    "COURSE_SEARCH_ID": "REPlACE_WITH_YOUR_KEY",
    "DEGREE_SEARCH_ID": "REPlACE_WITH_YOUR_KEY",
    "RANDOM_SEARCH_ID": "REPlACE_WITH_YOUR_KEY"
  }
```



In [1]:
import os
import sys
from dotenv import load_dotenv

import random
import string

#append the parent path to sys.path
sys.path.append(os.path.dirname(os.getcwd()))

from CompassUTD import CompassInference
from langchain.memory import ReadOnlySharedMemory, MongoDBChatMessageHistory, ConversationBufferMemory,ConversationBufferWindowMemory


load_dotenv()
agent = CompassInference()



## MongoDB Test

In [2]:
# Use MongoDB
connection_string = f"mongodb+srv://{str(os.getenv('MONGODB_LOGIN'))}@{str(os.getenv('MONGODB_LOCATION'))}"

random_letter_string = "Test_".join([random.choice(string.ascii_letters) for _ in range(11)])

message_history = MongoDBChatMessageHistory(
        connection_string=connection_string, session_id= random_letter_string
    )

In [6]:

memmory_store = ConversationBufferWindowMemory(memory_key="chat_history",chat_memory=message_history, k = 4)

read_only_memory = ReadOnlySharedMemory(memory=memmory_store)


#If you want to make a new chat query with same memory, replace the user_message with next message and run this cell
#user_message = "Who research AI at UTD?"
#user_message = "How hard is this class?"
user_message = "What is Math 2419"
#user_message = "What is the difference between math 2417 and 2419?"

bot_message = agent.run(user_message = user_message, 
                        read_only_memory= ReadOnlySharedMemory(memory=read_only_memory)
                        )

print(bot_message)

#Save chat history
message_history.add_user_message(user_message)
message_history.add_ai_message(bot_message)




> Entering new  chain...
Thought: I don't know what you are asking.
Action: general_search
Action Input: Math 2419
Observation: {'data': ''}
Thought:Final Answer: I don't know what you are asking.

> Finished chain.


I'm sorry, I don't know what you are asking. Can you try rephrasing your question? Go Comets!

In [4]:
from rich import print
print(
    message_history.messages
)

[
    HumanMessage(content='How hard is this class?', additional_kwargs={}, example=False),
    AIMessage(content='Not relevant', additional_kwargs={}, example=False)
]

In [5]:
print(
    agent.run(
        user_message = "What is the difference between Math 2417 and Math 2419",
        read_only_memory = ReadOnlySharedMemory(
            memory = read_only_memory
        )
    )
)



> Entering new  chain...
Thought: I need to know more about Math 2417 and Math 2419
Action: course_search
Action Input: Math 2417
Observation: {'items': [{'math2417': 'MATH2417 - Calculus IMATH 2417 Calculus I (4 semester credit hours) Functions, limits, continuity, differentiation; integration of function of one variable; logarithmic, exponential, and inverse trigonometric functions; techniques of integration, and applications. Three lecture hours and two discussion hours a week; problem section required with MATH 2417, and will also be registered for exam section. Not all MATH/STAT courses may be counted toward various degree plans. Please consult your degree plan to determine the appropriate MATH/STAT course requirements. Prerequisite: ALEKS score required or a grade of at least a C- in MATH 2306 or MATH 2312. (3-2) S'}, {}]}
Thought:Action: course_search
Action Input: Math 2419
Observation: {'items': [{'math2419': 'MATH2419 - Calculus IIMATH 2419 Calculus II (4 semester credit ho

Math 2417 is a calculus course that covers functions, limits, continuity, differentiation, integration, 
logarithmic, exponential, and inverse trigonometric functions. Math 2419 is a continuation of Math 2417 and covers 
improper integrals, sequences, infinite series, power series, parametric equations and polar coordinates, vectors, 
vector valued functions, functions of several variables, partial derivatives and applications, and multiple 
integration. Go Comets!

# Test

## Tell me the difference between a course

inputs = "What is the difference between Math 2417 and Math 2419"

bot_message = agent.run(
    user_message = inputs , 
    read_only_memory = 
)